Python 3.11 has about 45-50% improvement over previous version for simple calculations like fibonacci function

In [ ]:
%pip install -q tqdm pandas tabulate

In [15]:
script = """

import platform
import json
import os
import time

N = 30
n_sim = 100

def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

def bench(n):
    t1 = time.time()
    fibonacci(n)
    t2 = time.time()
    return t2-t1

results = []

for _ in range(n_sim):
    duration = bench(N)
    result = {"version": platform.python_version(), "n": N, "duration": duration}
    results.append(result)

print(json.dumps(obj=results, indent=4))
"""

with open("script.py", "w") as f:
    f.write(script)

In [42]:
import subprocess
from tqdm import tqdm

# List of Python versions you want to use
python_versions = [
"3.7",
"3.8", 
"3.9", 
"3.10",
"3.11",
"3.12",
]

# Loop through each Python version
for version in tqdm(python_versions):
    # Construct the Docker command
    command = (
        f"docker run --rm -v $(pwd):/usr/src/myapp -w /usr/src/myapp "
        f"python:{version}-slim python script.py > results_{version}.json"
    )

    # Execute the command
    subprocess.run(command, shell=True, check=True)


100%|██████████| 6/6 [03:23<00:00, 33.93s/it]


In [43]:
import subprocess

def get_python_version(python_major_version):
    try:
        command = f"docker run --rm python:{python_major_version}-slim python -c \"import platform; print(platform.python_version())\""
        
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True)
        
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        return None

latest = get_python_version(python_versions[-1])
print("latest:", latest)

latest: 3.12.1


In [44]:
import pandas as pd
import json

results = []
for version in python_versions:
    with open(f"results_{version}.json") as f:
        results.extend(json.load(f))
df = pd.DataFrame(results)

In [45]:
df_grouped = df[['version', 'duration']].groupby(["version"])['duration'].agg(['count','mean'])

latest_mean = df_grouped.loc[latest]['mean']
df_grouped['latest_improvement'] = 1 - latest_mean / df_grouped['mean']
df_grouped = df_grouped.reset_index()
df_grouped[['version_major', 'version_minor', 'version_patch']] = df_grouped['version'].str.split('.', expand=True).astype(int)
df_grouped = df_grouped.sort_values(['version_major', 'version_minor', 'version_patch'])\
    .drop(columns=['version_major', 'version_minor', 'version_patch'])\
    .reset_index(drop=True)
df_grouped

,version,count,mean,latest_improvement
0,3.7.17,100,0.425265,0.551687
1,3.8.18,100,0.399123,0.522323
2,3.9.18,100,0.406086,0.530513
3,3.10.13,100,0.390124,0.511304
4,3.11.7,100,0.188615,-0.010798
5,3.12.1,100,0.190652,0.000000


In [48]:
df_grouped.to_markdown("temp.md")